In [ ]:
!pip install torch
!pip install torchvision
!pip install torchaudio
!pip install torchtext
!pip install datasets
!pip install tokenizers
!pip install torchmetrics
!pip install tensorboard
!pip install altair
!pip install wandb
!pip install spacy
!pip install torchinfo
!pip install scikit-learn
!pip install langchain-core
!pip install langchain-community
!pip install langchain_groq
!pip install sentence-transformers
!pip install faiss-cpu
!pip install langchain_google_genai
!pip install langchain_anthropic
!pip install langchain_huggingface
!pip install langchain_openai
!pip install grandalf


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
#from langchain_anthropic import ChatAnthropic
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

In [ ]:
from google.colab import userdata
groq_api_key=userdata.get('groq_api_key')
HF_TOKEN=userdata.get('HF_TOKEN')
#print(groq_api_key)
#print(HF_TOKEN)
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
#print(OPENAI_API_KEY)
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
#print(GOOGLE_API_KEY)
langsmith=userdata.get('LANGSMITH_API_KEY')
#print(langsmith)

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
openai_model=ChatOpenAI(model="gpt-4o-mini",temperature=0)

In [ ]:
####prompt template use at run time and chat prompt template we can have list of templates
###list of messages for chatprompttemplate(System message, ai message, human message)
###define tuple over here

chat_template=ChatPromptTemplate(
    [
        ("system","you are a helpful {domain} expert"),
        ("human","explain the {topic} in simple terms")
    ]
)

In [ ]:
prompt=chat_template.invoke({"domain":"medical","topic":"maleria"})

In [ ]:
print(prompt)

In [ ]:
##Complete prompt   messages=[SystemMessage(content='you are a helpful medical expert', additional_kwargs={}, response_metadata={}),
#                             HumanMessage(content='explain the maleria in simple terms', additional_kwargs={}, response_metadata={})]

In [ ]:
openai_model.invoke(prompt).content

In [ ]:
prompt = chat_template.invoke({"domain":"education","topic":"AI"})

In [ ]:
print(prompt)

In [ ]:
openai_model.invoke(prompt)

#CHAINING
#Chaining using LCEL


In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
parser=StrOutputParser()

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
prompt=PromptTemplate(
    template="can you give me a detail explaination of {topic}",
    input_variables=['topic']

)

In [ ]:
prompt

In [ ]:
openai_model

#Simple LLM chain

In [ ]:
chain= prompt|openai_model|parser

In [ ]:
chain.invoke({"topic":"machine learning"})

In [ ]:
chain.get_graph().print_ascii()

#Sequential chain using LCEL

In [ ]:
prompt1=PromptTemplate(
    template="get a detail report on {topic}",
    input_variables=["topic"]
)

prompt2=PromptTemplate(
    template="generate a 3 point of summary from the following {text}",
    input_variables=["text"]
)

In [ ]:
text="""In November 2023, xAI began previewing Grok as a chatbot to selected people,[10] with participation in the early access program being limited to paid X Premium users.[11]

It was announced that once the bot was out of early beta, it would only be available to higher tier X Premium+ subscribers.[12]

At the time of the preview, xAI described the chatbot as "a very early beta product – the best we could do with 2 months of training" that could "improve rapidly with each passing week".[13]

On March 11, 2024, Musk posted on X that the language model would go open source within a week. Six days later, on March 17, Grok-1 was open sourced under the Apache-2.0 license.[14][15] Disclosed were the networks architecture and its weight parameters.[16]

On March 26, 2024, Musk announced that Grok would be enabled for premium subscribers, not just those on the higher-end tier, Premium+.[17]

Grok-1.5
Grok-1.5
Developer(s)	xAI
Initial release	May 15, 2024; 10 months ago
Predecessor	Grok-1.5
Successor	Grok-2
Type
Large language model
Foundation model
License	Proprietary
Website	x.ai/blog/grok-1.5
On March 29, 2024, Grok-1.5 was announced, with "improved reasoning capabilities" and a context length of 128,000 tokens.[18] Grok-1.5 was released to all X Premium users on May 15, 2024.[1]

On April 4, 2024, an update to X's "Explore" page included summaries of breaking news stories written by Grok, a task previously assigned to a human curation team.[19]

On April 12, 2024, Grok-1.5 Vision (Grok-1.5V) was announced. Grok-1.5V is able to process a wide variety of visual information, including documents, diagrams, graphs, screenshots, and photographs.[20] Grok-1.5V was never released to the public.

On May 4, 2024, Grok became available in the United Kingdom,[21] that being the only country in Europe to support Grok at the moment due to the impending Artificial Intelligence Act rules in the European Union. Grok was later reviewed by the EU and was released on May 16, 2024.[22]"""

In [ ]:
prompt1=PromptTemplate(
    template="analysis the the given text carefully {text} and take the necessary data",
    input_variables=["topic"]
)

prompt2=PromptTemplate(
    template="summarize the given text in 2 bullet points {text}",
    input_variables=["text"]
)

In [ ]:
openai_model

In [ ]:
parser

In [ ]:
chain = prompt1 | openai_model | parser | prompt2 | openai_model | parser

In [ ]:
chain.get_graph().print_ascii()

In [ ]:
result=chain.invoke({"text":text})

In [ ]:
print(result)

#Parallel Chain

In [ ]:
prompt1=PromptTemplate(
    template="generate simple summary from the following text \n {text}",
    input_variables=["text"]
)

prompt2=PromptTemplate(
    template="generate 3 question and answer from the following text \n {text}",
    input_variables=["text"]
    )

prompt3=PromptTemplate(
    template="analysis the summary and qa and generate the 5 important quiz with 4 possible answer \n summary: {summary}, Q&A: {qa}",
    input_variables=["summary","qa"]
)

In [ ]:
from langchain.schema.runnable import RunnableParallel

In [ ]:
parallel_chain=RunnableParallel({
    "summary": prompt1 | openai_model | parser,
    "qa" : prompt2 | openai_model | parser
}
)

In [ ]:
parallel_chain

In [ ]:
parallel_chain.invoke({"text":text})

In [ ]:
{'summary': 'Intelligent agents are a form of digital agency that actively pursues its goals, makes decisions, and takes actions over time. They vary in complexity, from basic control systems to complex human-like entities. These agents operate based on an objective function that represents their goals and plans actions to maximize its expected value. They are a topic of study in various fields, including artificial intelligence, economics, cognitive science, ethics, and philosophy. They can be described as abstract functional systems and are often closely related to software agents that perform tasks for users.\n\nIn other terms, intelligent agents are computer programs designed to independently achieve specific objectives, embodying a new level of digital agency by making proactive, goal-oriented decisions over time. They range from simple control mechanisms like thermostats to advanced, human-like systems. These agents are informed by an objective function representing their goals and work towards maximizing its value. They are not only pivotal in AI but also hold importance in fields like economics, cognitive science, philosophy, and ethics.',
 'qa': '1) What is an example of a simple intelligent agent?\nAnswer: A basic thermostat or control system is considered a simple intelligent agent.\n\n2) What encapsulates the goals of an intelligent agent and forms the basis for its operations?\nAnswer: An objective function encapsulates the goals of an intelligent agent and forms the basis for its operations. \n\n3) What are software agents in the field of artificial intelligence and how are they related to intelligent agents?\nAnswer: Software agents are autonomous computer programs that carry out tasks on behalf of users. They are closely related to intelligent agents, often performing tasks and decision making in a proactive manner.'}

In [ ]:
#Intelligent agents, also known as AI agents, are preemptive agents that perform certain actions to achieve predefined goals. They extend the concept of conventional agents by not only responding but also acting preemptively, making decisions, and taking actions over an extended period. These AI agents can range from simple systems like a thermostat to more complex beings like humans or even larger systems such as firms, states, or biomes.

#The behavior of these intelligent agents is driven by an objective function that encapsulates their targets. They function with the aim of maximizing the expected value of this objective function. Notable examples include a reinforcement learning agent guided by a reward function, and an evolutionary algorithm steered by a fitness function. The reward function for the reinforcement learning agent plays a crucial role by allowing programmers to shape its desired behavior. Similarly, the fitness function acts as the steering wheel for an evolutionary algorithm's behavior.

#The paradigms of intelligent agents are probed in various fields such as cognitive science, ethics, philosophy, and computer social simulations. These agents are often described in abstract terms, making them highly similar to computer programs. This schematic description often results in them being referred to as abstract intelligent agents or, borrowing a term from economics, "rational agents". Also, they have a close relation to software agents, which are autonomous computer programs that carry out tasks on behalf of users. Such software agents are another key application of the concept of intelligent agents

In [ ]:
merge_chain= prompt3 | openai_model | parser

In [ ]:
chain = parallel_chain | merge_chain

In [ ]:
chain.get_graph().print_ascii()

In [ ]:
text="""AI agent or simply agent), expands this concept by proactively pursuing goals, making decisions, and taking actions over extended periods, thereby exemplifying a novel form of digital agency.[1]

Intelligent agents can range from simple to highly complex. A basic thermostat or control system is considered an intelligent agent, as is a human being, or any other system that meets the same criteria—such as a firm, a state, or a biome.[2]

Intelligent agents operate based on an objective function, which encapsulates their goals. They are designed to create and execute plans that maximize the expected value of this function upon completion.[3] For example, a reinforcement learning agent has a reward function, which allows programmers to shape its desired behavior.[4] Similarly, an evolutionary algorithm's behavior is guided by a fitness function.[5]

Intelligent agents in artificial intelligence are closely related to agents in economics, and versions of the intelligent agent paradigm are studied in cognitive science, ethics, and the philosophy of practical reason, as well as in many interdisciplinary socio-cognitive modeling and computer social simulations.

Intelligent agents are often described schematically as abstract functional systems similar to computer programs. To distinguish theoretical models from real-world implementations, abstract descriptions of intelligent agents are called abstract intelligent agents. Intelligent agents are also closely related to software agents—autonomous computer programs that carry out tasks on behalf of users. They are also referred to using a term borrowed from economics: a "rational agent".[2]"""

In [ ]:
result=chain.invoke({"text":text})

In [ ]:
print(result)